In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm
import lightgbm as lgb
from sklearn.model_selection import KFold, StratifiedKFold
import warnings
from sklearn.metrics import f1_score, classification_report, mean_squared_error
import gc
import xgboost as xgb
from scipy import stats
import datetime
import time
from scipy.stats import entropy, kurtosis
import multiprocessing
from gensim.models.word2vec import LineSentence
from gensim.corpora import WikiCorpus
from gensim.models import Word2Vec
tqdm.pandas()

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

warnings.filterwarnings('ignore')

In [2]:
seed = 2020
n_fold = 5

In [3]:
df_train = pd.read_csv('raw_data/train.csv')
df_test = pd.read_csv('raw_data/test_noLabel.csv')
df_sub = pd.read_csv('raw_data/submit_examp.csv')

In [4]:
df_feature = pd.concat([df_train, df_test], sort=False)

In [5]:
df_feature.columns = ['ID', 'max_phase',
                      'weight', 'RO5', 'AlogP', 'Features', 'Label']

In [6]:
df_Features = df_feature[['ID', 'Features']].copy(deep=True)
df_Features['Features'] = df_feature['Features'].apply(lambda x: x[1:-1])
df_temp = df_Features['Features'].str.split(',', expand=True)

np_temp = df_temp.values
np_temp = np_temp.astype(np.float64)

df_temp = pd.DataFrame(np_temp)
df_temp.columns = ['Features_{}'.format(i) for i in range(df_temp.shape[1])]

df_feature = df_feature.reset_index(drop=True)
df_feature = pd.concat([df_feature, df_temp], axis=1)

del np_temp, df_temp, df_Features
gc.collect()

13

In [7]:
df_feature['weight'] = np.log1p(df_feature['weight'])
df_feature['AlogP'] = np.log1p(df_feature['AlogP'])

In [8]:
def func(x):
    a = x[1:-1].split(',')
    a = [float(i) for i in a]
    return a

df_feature['Features'] = df_feature['Features'].apply(func)

In [9]:
df_feature['Features_sum'] = df_feature['Features'].apply(lambda x: sum(x))
df_feature['Features_mean'] = df_feature['Features'].apply(
    lambda x: np.mean(x))
df_feature['Features_max'] = df_feature['Features'].apply(lambda x: max(x))
df_feature['Features_min'] = df_feature['Features'].apply(lambda x: min(x))

In [10]:
df_feature.head()

ID  max_phase    weight  RO5     AlogP  \
0  1003          0  0.187934  0.0  0.475016   
1  1819          0  0.122267  0.0  0.464896   
2  6090          0  0.150558  0.0  0.465597   
3  3916          0  0.106400  0.0  0.502077   
4  8480          0  0.149903  0.0  0.554738   

                                            Features      Label  Features_0  \
0  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 0.0, 10.0,...   3.190476         0.0   
1  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, ...   9.740969         0.0   
2  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, ...  10.545341         0.0   
3  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ...   3.206803         0.0   
4  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ...  10.950807         0.0   

   Features_1  Features_2  Features_3  Features_4  Features_5  Features_6  \
0         0.0         0.0         0.0         0.0         0.0         2.0   
1         0.0         0.0         0.0         0.0         0.0         0.0   
2         0.0         0.0         0.0         0.0         0.0         0.0   
3         0.0         0.0         0.0         0.0         0.0         1.0   
4         0.0         0.0         0.0         0.0         0.0         1.0   

   Features_7  Features_8  Features_9  Features_10  Features_11  Features_12  \
0         0.0        10.0         0.0          0.0          3.0          2.0   
1         0.0         5.0         0.0          0.0          5.0          1.0   
2         0.0         3.0         0.0          0.0          7.0          0.0   
3         0.0         0.0         0.0          0.0          9.0          0.0   
4         0.0         0.0         0.0          0.0         13.0          0.0   

   Features_13  Features_14  Features_15  Features_16  Features_17  \
0          0.0          0.0          1.0          1.0          2.0   
1          0.0          0.0          0.0          3.0          2.0   
2          0.0          0.0          0.0          4.0          2.0   
3          0.0          0.0          0.0          6.0          0.0   
4          0.0          0.0          0.0          4.0          4.0   

   Features_18  Features_19  Features_20  Features_21  Features_22  \
0          1.0          0.0          0.0          0.0          0.0   
1          0.0          0.0          1.0          0.0          0.0   
2          0.0          0.0          0.0          0.0          0.0   
3          0.0          0.0          0.0          0.0          0.0   
4          0.0          0.0          0.0          0.0          0.0   

   Features_23  Features_24  Features_25  Features_26  Features_27  \
0          1.0          0.0          0.0          0.0          0.0   
1          0.0          0.0          0.0          0.0          0.0   
2          0.0          0.0          0.0          0.0          0.0   
3          0.0          1.0          0.0          0.0          0.0   
4          1.0          0.0          0.0          0.0          0.0   

   Features_28  Features_29  Features_30  Features_31  Features_32  \
0          0.0          1.0          0.0          2.0          0.0   
1          4.0          0.0          0.0          1.0          0.0   
2          5.0          0.0          0.0          1.0          0.0   
3          1.0          0.0          0.0          0.0          0.0   
4          3.0          0.0          0.0          1.0          0.0   

   Features_33  Features_34  Features_35  Features_36  Features_37  \
0          1.0          2.0          1.0          0.0          1.0   
1          0.0          0.0          1.0          0.0          0.0   
2          0.0          1.0          0.0          1.0          0.0   
3          1.0          1.0          0.0          0.0          0.0   
4          0.0          0.0          1.0          0.0          0.0   

   Features_38  Features_39  Features_40  Features_41  Features_42  \
0          0.0          0.0          0.0          0.0          0.0   
1          0.0          0.0          0.0          0.0     

In [11]:
df_test = df_feature[df_feature['Label'].isnull()].copy()
df_train = df_feature[df_feature['Label'].notnull()].copy()

In [12]:
ycol = 'Label'
feature_names = list(
    filter(lambda x: x not in [ycol, 'Features'], df_train.columns))

model = lgb.LGBMRegressor(num_leaves=64,
                          max_depth=10,
                          learning_rate=0.01,
                          n_estimators=10000000,
                          subsample=0.8,
                          feature_fraction=0.8,
                          reg_alpha=0.5,
                          reg_lambda=0.5,
                          random_state=seed,
                          metric=None
                          )


oof = []
prediction = df_test[['ID']]
prediction['Label'] = 0
df_importance_list = []

kfold = KFold(n_splits=n_fold, shuffle=False, random_state=seed)
for fold_id, (trn_idx, val_idx) in enumerate(kfold.split(df_train[feature_names])):
    X_train = df_train.iloc[trn_idx][feature_names]
    Y_train = df_train.iloc[trn_idx][ycol]

    X_val = df_train.iloc[val_idx][feature_names]
    Y_val = df_train.iloc[val_idx][ycol]

    print('\nFold_{} Training ================================\n'.format(fold_id+1))

    lgb_model = model.fit(X_train,
                          Y_train,
                          eval_names=['train', 'valid'],
                          eval_set=[(X_train, Y_train), (X_val, Y_val)],
                          verbose=500,
                          eval_metric='rmse',
                          early_stopping_rounds=50)

    pred_val = lgb_model.predict(
        X_val, num_iteration=lgb_model.best_iteration_)
    df_oof = df_train.iloc[val_idx][['ID', ycol]].copy()
    df_oof['pred'] = pred_val
    oof.append(df_oof)

    pred_test = lgb_model.predict(
        df_test[feature_names], num_iteration=lgb_model.best_iteration_)
    prediction['Label'] += pred_test / n_fold

    df_importance = pd.DataFrame({
        'column': feature_names,
        'importance': lgb_model.feature_importances_,
    })
    df_importance_list.append(df_importance)

    del lgb_model, pred_val, pred_test, X_train, Y_train, X_val, Y_val
    gc.collect()


Fold_1 Training ================================

Training until validation scores don't improve for 50 rounds
[500]	train's rmse: 1.65908	valid's rmse: 2.46428
[1000]	train's rmse: 1.28532	valid's rmse: 2.28283
[1500]	train's rmse: 1.07178	valid's rmse: 2.21084
[2000]	train's rmse: 0.926694	valid's rmse: 2.17385
[2500]	train's rmse: 0.819757	valid's rmse: 2.15741
[3000]	train's rmse: 0.742002	valid's rmse: 2.15008
[3500]	train's rmse: 0.680441	valid's rmse: 2.14516
[4000]	train's rmse: 0.627123	valid's rmse: 2.14193
Early stopping, best iteration is:
[4195]	train's rmse: 0.609462	valid's rmse: 2.14105

Fold_2 Training ================================

Training until validation scores don't improve for 50 rounds
[500]	train's rmse: 1.71288	valid's rmse: 2.33921
[1000]	train's rmse: 1.29056	valid's rmse: 2.15079
[1500]	train's rmse: 1.08556	valid's rmse: 2.0918
[2000]	train's rmse: 0.9495	valid's rmse: 2.06627
[2500]	train's rmse: 0.848799	valid's rmse: 2.05288
[3000]	train's rmse: 0.7

In [13]:
df_importance = pd.concat(df_importance_list)
df_importance = df_importance.groupby(['column'])['importance'].agg(
    'mean').sort_values(ascending=False).reset_index()
df_importance

,column,importance
0,ID,24753.8
1,AlogP,6216.2
2,Features_3166,5220.4
3,Features_3157,4707.8
4,Features_3154,4703.4
5,Features_3152,4614.4
6,Features_3153,4148.8
7,Features_3156,4065.4
8,weight,3265.8
9,Features_3161,2939.8


In [14]:
df_oof = pd.concat(oof)
rmse = np.sqrt(mean_squared_error(df_oof[ycol], df_oof['pred']))
print('rmse:', rmse)

rmse: 2.0377320896192135


In [15]:
sub = prediction.copy(deep=True)
sub.to_csv('sub/{}.csv'.format(rmse), index=False, encoding='utf-8')

In [16]:
sub.head()

,ID,Label
6924,0,10.788361
6925,7,2.169278
6926,21,9.137595
6927,24,9.856023
6928,25,9.655334
